Contained within this notebook is a sample implementation and demo of the proposed **Fuzzy Conservative Q-Learning** procedure. The code is not necessarily optimal or efficient with respect to performance, but was more or less written for interpretability. Some functions that may be difficult to follow, such as ECM, actually should have a one-to-one correspondence with the original paper's notation (in the case of ECM, that would be the dynamic evolving neuro fuzzy system called DENFIS). 

In [ ]:
# the following can be installed without pip install

import os
import sys
import gym
import time
import torch
import random
import warnings
import numpy as np
import pandas as pd
import torch.nn as nn
import matplotlib.pyplot as plt

from torch import optim
from datetime import datetime
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from torch.nn.parameter import Parameter
from scipy.spatial.distance import minkowski  # for Evolving Clustering Method (ECM)
from sklearn.metrics import mean_squared_error  # for neuro-fuzzy network core functionality
from torch.utils.tensorboard import SummaryWriter
from sklearn.model_selection import train_test_split

In [ ]:
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning)

There are a few libraries we need to pip install.

In [ ]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1

In [ ]:
# we need to pip install d3rlpy (deep reinforcement learning) library

!pip install d3rlpy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.2 MB 13.0 MB/s 
     |████████████████████████████████| 125 kB 49.7 MB/s 
     |████████████████████████████████| 58 kB 6.2 MB/s 


In [ ]:
from d3rlpy.datasets import get_cartpole

The first algorithm we need to define is **Categorical Learning Induced Partitioning (CLIP)**. CLIP is used to generate the Gaussian membership functions that describe fuzzy sets. By running CLIP over the entire training data, we will be able to create a global language (i.e. linguistic terms) for interpretation.

In [ ]:
def gaussian(x, center, sigma):
    return np.exp(-1.0 * (np.power(x - center, 2) / np.power(sigma, 2)))

def R(sigma_1, sigma_2):
    # regulator function
    return (1/2) * (sigma_1 + sigma_2)

def CLIP(X, mins, maxes, terms=[], eps=0.2, kappa=0.6, theta=1e-8):
    # theta is a parameter I add to accomodate for the instance in which an observation has values that are the minimum/maximum
    # otherwise, when determining the Gaussian membership, a division by zero will occur
    # it essentially acts as an error tolerance
    antecedents = terms
    min_values_per_feature_in_X = mins
    max_values_per_feature_in_X = maxes
    for idx, x in enumerate(X):
        if not terms:
            # no fuzzy clusters yet, create the first fuzzy cluster
            for p in range(len(x)):
                c_1p = x[p]
                min_p = min_values_per_feature_in_X[p]
                max_p = max_values_per_feature_in_X[p]
                left_width = np.sqrt(-1.0 * (np.power((min_p - x[p]) + theta, 2) / np.log(eps)))
                right_width = np.sqrt(-1.0 * (np.power((max_p - x[p]) + theta, 2) / np.log(eps)))
                sigma_1p = R(left_width, right_width)
                terms.append([{'center': c_1p, 'sigma': sigma_1p, 'support':1}])
        else:
            # calculate the similarity between the input and existing fuzzy clusters
            for p in range(len(x)):
                SM_jps = []
                for j, A_jp in enumerate(terms[p]):
                    SM_jp = gaussian(x[p], A_jp['center'], A_jp['sigma'])
                    SM_jps.append(SM_jp)
                j_star_p = np.argmax(SM_jps)

                if np.max(SM_jps) > kappa:
                    # the best matched cluster is deemed as being able to give satisfactory description of the presented value
                    A_j_star_p = terms[p][j_star_p]
                    A_j_star_p['support'] += 1
                else:
                    # a new cluster is created in the input dimension based on the presented value
                    jL_p = None
                    jR_p = None
                    jL_p_differences = []
                    jR_p_differences = []
                    for j, A_jp in enumerate(terms[p]):
                        c_jp = A_jp['center']
                        if c_jp >= x[p]:
                            continue  # the newly created cluster has no immediate left neighbor
                        else:
                            jL_p_differences.append(np.abs(c_jp - x[p]))
                    try:
                        jL_p = np.argmin(jL_p_differences)
                    except ValueError:
                        jL_p = None

                    for j, A_jp in enumerate(terms[p]):
                        c_jp = A_jp['center']
                        if c_jp <= x[p]:
                            continue  # the newly created cluster has no immediate right neighbor
                        else:
                            jR_p_differences.append(np.abs(c_jp - x[p]))
                    try:
                        jR_p = np.argmin(jR_p_differences)
                    except ValueError:
                        jR_p = None

                    new_c = x[p]
                    new_sigma = None

                    # --- this new fuzzy set has no left or right neighbor ---
                    if jL_p is None and jR_p is None:
                        continue

                    # --- there is no left neighbor to this new fuzzy set ---
                    if jL_p is None:
                        cR_jp = terms[p][jR_p]['center']
                        sigma_R_jp = terms[p][jR_p]['sigma']
                        left_sigma_R = np.sqrt(-1.0 * (np.power(cR_jp - x[p], 2) / np.log(eps)))
                        sigma_R = R(left_sigma_R, sigma_R_jp)

                        new_sigma = sigma_R
                        # update the existing term to make room for the new term
                        terms[p][jR_p]['sigma'] = new_sigma

                    # --- there is no right neighbor to this new fuzzy set ---
                    elif jR_p is None:
                        cL_jp = terms[p][jL_p]['center']
                        sigma_L_jp = terms[p][jL_p]['sigma']
                        left_sigma_L = np.sqrt(-1.0 * (np.power(cL_jp - x[p], 2) / np.log(eps)))
                        sigma_L = R(left_sigma_L, sigma_L_jp)

                        new_sigma = sigma_L
                        # update the existing term to make room for the new term
                        terms[p][jL_p]['sigma'] = new_sigma

                    # --- there is BOTH a left and a right neighbor to this fuzzy set ---
                    else:
                        cR_jp = terms[p][jR_p]['center']
                        sigma_R_jp = terms[p][jR_p]['sigma']
                        left_sigma_R = np.sqrt(-1.0 * (np.power(cR_jp - x[p], 2) / np.log(eps)))
                        sigma_R = R(left_sigma_R, sigma_R_jp)

                        cL_jp = terms[p][jL_p]['center']
                        sigma_L_jp = terms[p][jL_p]['sigma']
                        left_sigma_L = np.sqrt(-1.0 * (np.power(cL_jp - x[p], 2) / np.log(eps)))
                        sigma_L = R(left_sigma_L, sigma_L_jp)

                        new_sigma = R(sigma_R, sigma_L)
                        # update the existing terms to make room for the new term
                        terms[p][jR_p]['sigma'] = terms[p][jL_p]['sigma'] = new_sigma
                    terms[p].append({'center':new_c, 'sigma':new_sigma, 'support':1})
    return terms

Evolving Clustering Method is required for fuzzy logic rule generation. Below we define its necessary functions.

In [ ]:
SUPPRESS_EXCEPTIONS = True

class Cluster:
    def __init__(self, center, radius):
        self.center = center
        self.radius = radius
        self.support = 1
        
    def add_support(self):
        self.support += 1
        
        
def general_euclidean_distance(x, y):
    if len(x) == len(y): 
        q = len(x)
        return minkowski(x, y, p=2) / np.power(q, 0.5)
    else:
        raise TypeError('The vectors must of of equal dimensionality in order to use the General Euclidean Distance metric.')


def ECM(X, Cs, Dthr, SUPPRESS_EXCEPTIONS=True):
    for i, x in enumerate(X):
        if len(Cs) == 0:
            """
            Step 0: Create the first cluster by simply taking the
            position of the first example from the input stream as the
            first cluster center Cc_{1}^{0}, and setting a value 0 for its cluster
            radius Ru_{1} [Fig. 2(a)].
            """
            C = Cluster(center=x, radius=0)
            Cs.append(C)
            
        """
        Step 1: If all examples of the data stream have been processed, the algorithm 
        is finished. Else, the current input example, $x_i$, is taken and the distances 
        between this example and all $n$ already created cluster 
        centers Cc_j, D_{ij} = ||x_{i} - Cc_{j}||, j = 1, 2, ..., n, are calculated.  
        """
        
        D_i = {}  # distances between the i'th 'x' and the centers for each j'th cluster; dictionary is indexed by j
        for j, C in enumerate(Cs):
            D_i[j] = general_euclidean_distance(x, C.center)
            
            """
            Step 2: If there is any distance value, D_{ij} = ||x_{i} - Cc_{j}||, equal to, 
            or less than, at least one of the radii, Ru_{j}, j = 1, 2, ..., n, it means 
            that the current example belongs to a cluster C_{m} with the minimum distance
            
            D_{im} = ||x_{i} - Cc_{m}|| = min(||x_{i} - Cc_{j}||)
            
            subject to the constraint D_{ij} < Ru_{j},   j = 1, 2, ..., n.
            
            In this case, neither a new cluster is created, nor are any existing clusters 
            updated (the cases of $x_4$ and $x_6$ in Fig. 2); the algorithm returns to Step 1. 
            Else—go to the next step.
            """
            if D_i[j] < C.radius:
                C.add_support()
                break  # the observation belongs to this cluster, C. Return to Step 1.
                
        if D_i[j] < C.radius:
            continue  # the observation belongs to this cluster, C. Return to Step 1.
        
        """ 
        Step 3: Find cluster (with center Cc_{a} and cluster radius Ru_{a}) from all existing cluster 
        centers through calculating the values S_{ij} = D_{ij} + Ru_{j}, j = 1, 2, ..., n, and then 
        choosing the cluster center with the minimum value S_{ia}:
            
            S_{ia} = D_{ia} + Ru_{a} = min(S_{ij}), j = 1, 2, ..., n.
        """
        
        S_i = {}
        for item in D_i.items():
            j = item[0]
            D_j = item[1]
            C_j = Cs[j]
            Ru_j = C_j.radius
            S_i[j] = D_j + Ru_j
        a = min(S_i, key=S_i.get)
        S_ia = S_i[a]
        
        """
        Step 4: If S_{ia} is greater than $2 * Dthr$, the example $x_i$ does not belong to any 
        existing clusters. A new cluster is created in the same way as described in Step 0 
        (the cases of $x_3$ and $x_8$ in Fig. 2), and the algorithm returns to Step 1.
        """
        
        if S_ia > (2.0 * Dthr):
            """
            Step 0: Create the first cluster by simply taking the
            position of the first example from the input stream as the
            first cluster center Cc_{1}^{0}, and setting a value 0 for its cluster
            radius Ru_{1} [Fig. 2(a)].
            """
            C = Cluster(center=x, radius=0)
            Cs.append(C)
            continue  # terminate further execution of this iteration
        else:
            """
            Step 5: If S_{ia} is not greater than $2 * Dthr$, the cluster $C_{a}$ is updated by moving 
            its center, Cc_{a}, and increasing the value of its radius, Ru_{a}. The updated 
            radius Ru_{a}^{new} is set to be equal to S_{ia} / 2 and the new center Cc_{a}^{new} is located
            at the point on the line connecting the $x_i$ and Cc_{a}, and the distance from the new 
            center Cc_{a}^{new} to the point $x_{i}$ is equal to Ru_{a}^{new} 
            (the cases of $x_2$, $x_5$, $x_7$ and $x_9$ in Fig. 2).
            The algorithm returns to Step 1
            """
            Ca = Cs[a]
            Ca.radius = S_ia / 2.0
            Ca.add_support()            
            n = Ca.support
            
            if n == 0:
                m_n_minus_1 = 0
            else:
                m_n_minus_1 = Ca.center
                
            # keep a running mean approximation of the cluster center
            
            Ca.center = ((n - 1) * Ca.center + x) / n
            
            if not SUPPRESS_EXCEPTIONS and general_euclidean_distance(Ca.center, x) != Ca.radius:
                raise Exception('The distance from the center of the relevant cluster is meant to be equal to the cluster\'s radius.')
    return Cs

Next, after ECM, is the Wang-Mendel method for fuzzy logic rule generation. It comes with a few extras that go beyond just the Wang-Mendel method as described in the paper because it is a general implementation. Specifically, it includes certainty factor calculations as described in the HyFIS paper. These certainty factors are ignored in our current procedure.

In [ ]:
def rule_creation(X, antecedents, existing_rules=[], existing_weights=[], consistency_check=True):
    start = time.time()
    rules = existing_rules
    weights = existing_weights
    for x in X:
        CF = 1.0 # certainty factor of this rule
        # this block of code is for antecedents
        A_star_js = []
        for p in range(len(x)):
            SM_jps = []
            for j, A_jp in enumerate(antecedents[p]):
                SM_jp = gaussian(x[p], A_jp['center'], A_jp['sigma'])
                SM_jps.append(SM_jp)
            CF *= np.max(SM_jps)
            j_star_p = np.argmax(SM_jps)
            A_star_js.append(j_star_p)

        # with some work, you can remove the 'C' key-value here in the rule, it stands for 'consequent(s)'
        # however, later on, in the neuro-fuzzy Q-network, it will expect this to be here
        R_star = {'A':A_star_js, 'C': [0], 'CF': CF, 'time_added': start}

        if not rules:  # no rules in knowledge base yet
            rules.append(R_star)
            weights.append(1.0)
        else:  # there are rules in the knowledge base, so check for uniqueness (i.e., this new rule you made -- R_star -- is it needed?)
            add_new_rule = True
            for k, rule in enumerate(rules):
                try:
                    if (rule['A'] == R_star['A']) and (rule['C'] == R_star['C']):
                        # the generated rule is not unique, it already exists, enhance this rule's weight
                        weights[k] += 1.0
                        rule['CF'] = min(rule['CF'], R_star['CF'])
                        add_new_rule = False
                        break
                except ValueError:  # this happens because R_star['A'] and R_star['C'] are Numpy arrays
                    if all(rule['A'] == list(R_star['A'])) and all(rule['C'] == list(R_star['C'])):
                        # the generated rule is not unique, it already exists, enhance this rule's weight
                        weights[k] += 1.0
                        rule['CF'] = min(rule['CF'], R_star['CF'])
                        add_new_rule = False
                        break
                    elif all(rule['A'] == list(R_star['A'])):  # my own custom else-if statement
                        if rule['CF'] <= R_star['CF']:
                            add_new_rule = False
            if add_new_rule:
                rules.append(R_star)
                weights.append(1.0)

    # check for consistency
    if consistency_check:
        all_antecedents = [rule['A'] for rule in rules]
    
        repeated_rule_indices = set()
        for k in range(len(rules)):
            indices = np.where(np.all(all_antecedents == np.array(rules[k]['A']), axis=1))[0]
            if len(indices) > 1:
                if len(repeated_rule_indices) == 0:  # this can be combined with the following elif-statement
                    repeated_rule_indices.add(tuple(indices))
                elif len(repeated_rule_indices) > 0:  # this can be combined with the above if-statement
                    repeated_rule_indices.add(tuple(indices))
    
        for indices in repeated_rule_indices:
            weights_to_compare = [weights[idx] for idx in indices]
            strongest_rule_index = indices[np.argmax(weights_to_compare)]  # keep the rule with the greatest weight to it
            for index in indices:
                if index != strongest_rule_index:
                    rules[index] = None
                    weights[index] = None
        rules = [rules[k] for k, rule in enumerate(rules) if rules[k] is not None]
        weights = [weights[k] for k, weight in enumerate(weights) if weights[k] is not None]
    
        # need to check that no antecedent terms are "orphaned" (i.e., they go unused)
    
        all_antecedents = [rule['A'] for rule in rules]
        all_antecedents = np.array(all_antecedents)
        for p in range(len(x)):
            if len(antecedents[p]) == len(np.unique(all_antecedents[:,p])):
                continue
            else:
                # orphaned antecedent term
                indices_for_antecedents_that_are_used = set(all_antecedents[:,p])
                updated_indices_to_map_to = list(range(len(indices_for_antecedents_that_are_used)))
                antecedents[p] = [antecedents[p][index] for index in indices_for_antecedents_that_are_used]
    
                paired_indices = list(zip(indices_for_antecedents_that_are_used, updated_indices_to_map_to))
                for index_pair in paired_indices:  # the paired indices are sorted w.r.t. the original indices
                    original_index = index_pair[0]  # so, when we updated the original index to its new index
                    new_index = index_pair[1]  # we are guaranteed not to overwrite the last updated index
                    all_antecedents[:,p][all_antecedents[:,p] == original_index] = new_index
    
        # update the rules in case any orphaned terms occurred
        for idx, rule in enumerate(rules):
            rule['A'] = all_antecedents[idx]

    return antecedents, rules, weights

The above three methods are now combined together into a single function called `unsupervised()`.

In [ ]:
def unsupervised(train_X, ecm=True, Dthr=1e-3, verbose=False):
    """
    Applies CLIP, ECM and Wang-Mendel method to produce the fuzzy sets, candidates and fuzzy logic rules, respectively.

    Parameters
    ----------
    train_X : 2-D Numpy array
        The input vector, has a shape of (number of observations, number of inputs/attributes).
    ecm : boolean, optional
        This boolean controls whether to enable the ECM algorithm for candidate rule generation. The default is True.
    Dthr : float, optional
        The distance threshold for the ECM algorithm; only matters if ECM is enabled. The default is 1e-3.
    verbose : boolean, optional
        If enabled (True), the execution of this function will print out step-by-step to show progress. The default is False.

    Returns
    -------
    The fuzzy logic rules, their corresponding weights (unused), and the antecedent terms they are defined over.

    """
    print('The shape of the training data is: (%d, %d)\n' %
          (train_X.shape[0], train_X.shape[1]))
    train_X_mins = train_X.min(axis=0)
    train_X_maxes = train_X.max(axis=0)

    if verbose:
        print('Creating/updating the membership functions...')

    eps = 0.2
    kappa = 0.6

    start = time.time()
    antecedents = CLIP(train_X, train_X_mins, train_X_maxes,
                       [], eps=eps, kappa=kappa)
    end = time.time()
    if verbose:
        print('membership functions for the antecedents generated in %.2f seconds.' % (
                end - start))

    if ecm:
        if verbose:
            print('\nReducing the data observations to clusters using ECM...')
        start = time.time()
        clusters = ECM(train_X, [], Dthr)
        if verbose:
            print('%d clusters were found with ECM from %d observations...' % (
                len(clusters), train_X.shape[0]))
        reduced_X = [cluster.center for cluster in clusters]
        end = time.time()
        if verbose:
            print('done; the ECM algorithm completed in %.2f seconds.' %
                  (end - start))
    else:
        reduced_X = train_X

    if verbose:
        print('\nCreating/updating the fuzzy logic rules...')
    start = time.time()
    antecedents, rules, weights = rule_creation(reduced_X, antecedents, [], [],
                                                             consistency_check=False)

    K = len(rules)
    end = time.time()
    if verbose:
        print('%d fuzzy logic rules created/updated in %.2f seconds.' %
              (K, end - start))
    return rules, weights, antecedents

A Gaussian activation function is then defined where it can have different centers and widths depending on which input variable and input term it is describing. For example, the i'th fuzzy set (across all input dimensions) would be defined by its corresponding i'th center and sigma.

In [ ]:
class Gaussian(nn.Module):
    """
    Helpful reference: https://towardsdatascience.com/extending-pytorch-with-custom-activation-functions-2d8b065ef2fa

    Implementation of the Gaussian membership function.
    Shape:
        - Input: (N, *) where * means, any number of additional
          dimensions
        - Output: (N, *), same shape as the input
    Parameters:
        - centers: trainable parameter
        - sigmas: trainable parameter
    Examples:
        # >>> a1 = gaussian(256)
        # >>> x = torch.randn(256)
        # >>> x = a1(x)
    """

    def __init__(self, in_features, centers=None, sigmas=None, trainable=True):
        """
        Initialization.
        INPUT:
            - in_features: shape of the input
            - centers: trainable parameter
            - sigmas: trainable parameter
            centers and sigmas are initialized randomly by default,
            but sigmas must be > 0
        """
        super(Gaussian, self).__init__()
        self.in_features = in_features

        # initialize centers
        if centers is None:
            self.centers = Parameter(torch.randn(self.in_features))
        else:
            self.centers = torch.tensor(centers)

        # initialize sigmas
        if sigmas is None:
            self.sigmas = Parameter(torch.abs(torch.randn(self.in_features)))
        else:
            # make sure the sigma values are positive
            self.sigmas = torch.abs(torch.tensor(sigmas))

        self.centers.requires_grad = trainable
        self.sigmas.requiresGrad = trainable
        self.centers.grad = None
        self.sigmas.grad = None

    def forward(self, x):
        """
        Forward pass of the function. Applies the function to the input elementwise.
        """

        return torch.exp(-1.0 * (torch.pow(x - self.centers, 2) / torch.pow(self.sigmas, 2)))

    The terms 'Fuzzy Logic Controller' and 'Neuro-Fuzzy Network' are used interchangeably in this code.

In [ ]:
class FLC(nn.Module):
    """
    Helpful reference: https://towardsdatascience.com/extending-pytorch-with-custom-activation-functions-2d8b065ef2fa

    Implementation of the Fuzzy Logic Controller.
    Shape:
        - Input: (N, *) where * means, any number of additional
          dimensions
        - Output: (N, *), same shape as the input
    Parameters:
        - centers: trainable parameter
        - sigmas: trainable parameter
    Examples:
        # >>> antecedents = [[{'type': 'gaussian', 'parameters': {'center': 1.2, 'sigma': 0.1}},
                            {'type': 'gaussian', 'parameters': {'center': 3.0, 'sigma': 0.4}}],
                            [{'type': 'gaussian', 'parameters': {'center': 0.2, 'sigma': 0.4}}]]
        # consequences are not required, default is None
        # >>> consequences = [[{'type': 'gaussian', 'parameters': {'center': 0.1, 'sigma': 0.7}},
                            {'type': 'gaussian', 'parameters': {'center': 0.4, 'sigma': 0.41}}],
                            [{'type': 'gaussian', 'parameters': {'center': 0.9, 'sigma': 0.32}}]]
        # if consequences are not to be specified, leave the key-value out
        # >>> rules = [{'antecedents':[0, 0], 'consequences':[0]}, {'antecedents':[1, 0], 'consequences':[1]}]
        # >>> n_input = len(antecedents)  # the length of antecedents should be equal to number of inputs
        # >>> n_output = len(consequences)  # the length of antecedents should be equal to number of inputs
        # >>> flc = FLC(n_input, n_output, antecedents, rules, consequences)
        # >>> x = torch.randn(n_input)
        # >>> y = flc(x)
    """

    def __init__(self, in_features, out_features, antecedents, rules, consequences=None):
        """
        Initialization.
        INPUT:
            - in_features: shape of the input
            - consequences: (optional) trainable parameter
            consequences are initialized randomly by default,
            but sigmas must be > 0
        """
        super(FLC, self).__init__()
        self.in_features = in_features

        # find the number of antecedents per input variable
        num_of_antecedents = np.zeros(in_features).astype('int32')
        unique_id = 0
        gaussians = {'centers': [], 'sigmas': []}  # currently, we assume only Gaussians are used
        self.input_variable_ids = []
        self.transformed_x_length = 0
        for input_variable_idx in range(in_features):
            num_of_antecedents[input_variable_idx] = len(antecedents[input_variable_idx])
            self.input_variable_ids.append(set())
            for term_idx, antecedent in enumerate(antecedents[input_variable_idx]):
                try:
                    gaussians['centers'].append(antecedent['parameters']['center'])
                    gaussians['sigmas'].append(antecedent['parameters']['sigma'])
                except KeyError:
                    gaussians['centers'].append(antecedent['center'])
                    gaussians['sigmas'].append(antecedent['sigma'])
                antecedent['id'] = unique_id
                self.input_variable_ids[-1].add(unique_id)
                unique_id += 1
        self.transformed_x_length = unique_id

        # find the total number of antecedents across all input variables
        n_rules = len(rules)
        self.links_between_antecedents_and_rules = np.zeros((num_of_antecedents.sum(), n_rules))

        for rule_idx, rule in enumerate(rules):
            try:
                for input_variable_idx, term_idx in enumerate(rule['antecedents']):
                    new_term_idx = antecedents[input_variable_idx][term_idx]['id']
                    self.links_between_antecedents_and_rules[new_term_idx, rule_idx] = 1
            except KeyError:
                for input_variable_idx, term_idx in enumerate(rule['A']):
                    new_term_idx = antecedents[input_variable_idx][term_idx]['id']
                    self.links_between_antecedents_and_rules[new_term_idx, rule_idx] = 1

        # begin creating the model's layers
        self.input_terms = Gaussian(in_features=self.in_features, centers=gaussians['centers'],
                                    sigmas=gaussians['sigmas'], trainable=False)

        # initialize consequences
        if consequences is None:
            num_of_consequent_terms = len(rules)
            self.consequences = Parameter(torch.zeros(num_of_consequent_terms, out_features))
        else:
            self.consequences = Parameter(torch.tensor(consequences))

        self.consequences.requires_grad = True

    def __transform(self, X):
        """
        Transforms the given 'X' to make it compatible with the first layer.

        The shape of 'X' is (num. of observations, num. of features).
        """
        shape = X.shape
        n_observations = shape[0]  # number of observations
        new_X = np.zeros((n_observations, self.transformed_x_length))
        for input_variable_idx, indices_to_repeat_for in enumerate(self.input_variable_ids):
            min_column_idx = min(indices_to_repeat_for)
            max_column_idx = max(indices_to_repeat_for) + 1
            copies = len(indices_to_repeat_for)  # how many copies we should make of this column
            new_X[:, min_column_idx:max_column_idx] = np.repeat(X[:, input_variable_idx], copies).reshape(
                (n_observations, copies))
        return torch.tensor(new_X)  # the shape of new_X should now be (num. of observations, num. of antecedent terms)

    def forward(self, X):
        """
        Forward pass of the function. Applies the function to the input elementwise.

        The shape of 'X' is (num. of observations, num. of features).
        """
        # we need to make the given 'X' compatible with our first layer,
        # which means repeating it for some entries
        antecedents_memberships = self.input_terms(self.__transform(X))
        terms_to_rules = antecedents_memberships[:, :, None] * torch.tensor(self.links_between_antecedents_and_rules)
        terms_to_rules[terms_to_rules == 0] = 1.0  # ignore zeroes, this is from the weights between terms and rules
        # the shape of terms_to_rules is (num of observations, num of ALL terms, num of rules)
        rules_applicability = terms_to_rules.prod(dim=1)
        numerator = (rules_applicability * self.consequences.T[0]).sum(dim=1)  # MISO
        denominator = rules_applicability.sum(dim=1)
        denominator[denominator == 0] = 1e-300  # add a minimum rule applicability to avoid division by zero error
        return numerator / denominator  # the dim=1 is taking product across ALL terms, now shape (num of observations, num of rules), MISO

    def predict(self, X):
        """
        A wrapper function, for calls that prefer the usage of 'predict'.

        The shape of 'X' is (num. of observations, num. of features).
        """
        return self.forward(X)

The above created FLC class can only handle multi-input-single-output (MISO). In other words, we could only calculate the Q-value of a single possible action. However, to extend this to multiple actions (i.e., multiple-output), we use the well-known fact that a collection of multiple FLCs is commonly used for producing multiple outputs.

In [ ]:
class MultiFLC:
    """
    A multi-input-multi-output (MIMO) Neuro-Fuzzy Network is a collection of
    multiple multi-input-single-output (MISO) Neuro-Fuzzy Networks (Klir, 1992).

    Essentially, for each possible action, we create a MISO Neuro-Fuzzy Q-Network
    to learn that action's Q-values across all the fuzzy logic rules.
    """
    def __init__(self, n_inputs, n_outputs, antecedents, rules, learning_rate=3e-4, cql_alpha=0.5):
        """
        Build the MIMO Neuro-Fuzzy Q-Network with an Adam optimizer for each individual FLC (per action).
        """
        self.flcs = []
        self.optimizers = []
        self.cql_alpha = cql_alpha
        for flc_idx in range(n_outputs):
            flc = FLC(n_inputs, 1, antecedents, rules)
            self.flcs.append(flc)
            # https://stackoverflow.com/questions/42966393/is-it-good-learning-rate-for-adam-method
            self.optimizers.append(optim.Adam(flc.parameters(), lr=learning_rate))

    def predict(self, X):
        """
        Using all Neuro-Fuzzy networks, output all Q-values using the entire MIMO Neuro-Fuzzy Q-network.

        The shape of 'X' is (num. of observations, num. of features).
        """
        output = []
        for flc in self.flcs:
            output.append(list(flc.predict(X).detach().numpy()))
        return torch.tensor(output).T

    def train(self, mode):
        """
        Disable training for all Neuro-Fuzzy networks used in this MIMO Neuro-Fuzzy Q-network.
        """
        for flc in self.flcs:
            flc.train(mode)

    def zero_grad(self):
        """
        Zeroes the gradient for each Neuro-Fuzzy network that creates this MIMO Neuro-Fuzzy Q-network.
        """
        for flc in self.flcs:
            flc.zero_grad()

    def fcql_loss_function(self, all_q_values, pred_qvalues, target_qvalues, action_indices, flc_idx):
        """
        The Fuzzy Conservative Q-Learning loss function.
        To ensure correct implementation, this code is an adaptation of the loss function provided from:

        https://colab.research.google.com/drive/1oJOYlAIOl9d1JjlutPY66KmfPkwPCgEE?usp=sharing
        """
        logsumexp_qvalues = torch.logsumexp(all_q_values, dim=-1)

        tmp_pred_qvalues = all_q_values.gather(
            1, action_indices.reshape(-1, 1)).squeeze()
        cql_loss = logsumexp_qvalues - tmp_pred_qvalues

        new_targets = target_qvalues[:, flc_idx]
        loss = torch.mean((pred_qvalues - new_targets) ** 2)
        return loss + self.cql_alpha * torch.mean(cql_loss)

    def offline_update(self, states, target_q_values, action_indices):
        """
        Updates each individual FLC's Q-values.
        """
        self.train(True)  # make sure training is enabled
        avg_loss = 0.
        all_q_values = self.predict(states)
        # the loss must be computed for each individual FLC
        for flc_idx, flc in enumerate(self.flcs):
            # compute the loss and its gradients
            q_values = flc.predict(states)  # get the Q-values for this action using its corresponding FLC
            loss = self.fcql_loss_function(all_q_values, q_values, target_q_values, action_indices, flc_idx)
            loss.backward()

            # apply the update
            self.optimizers[flc_idx].step()
            avg_loss += loss.item()
        self.train(False)  # when not training, turn it off
        return avg_loss / len(self.flcs)

    def compute_loss(self, states, target_q_values, action_indices):
        """
        Only compute the loss, but do not apply an update.
        """
        self.train(False)  # just in case, disable training
        avg_loss = 0.
        all_q_values = self.predict(states)
        for flc_idx, flc in enumerate(self.flcs):
            q_values = flc.predict(states)  # get the Q-values for this action using its corresponding FLC
            loss = self.fcql_loss_function(all_q_values, q_values, target_q_values, action_indices, flc_idx)
            avg_loss += loss.item()
        self.train(True)  # activate training
        return avg_loss / len(self.flcs)

The following code block comes from the `d3rlpy` library, but it has been modified to return both the average and standard deviation of the online evaluation.

In [ ]:
def evaluate_on_environment(env, n_trials=100, epsilon=0.0, render=False):
    """ Returns scorer function of evaluation on environment.

    This function returns scorer function, which is suitable to the standard
    scikit-learn scorer function style.
    The metrics of the scorer function is ideal metrics to evaluate the
    resulted policies.

    .. code-block:: python

        import gym

        from d3rlpy.algos import DQN
        from d3rlpy.metrics.scorer import evaluate_on_environment


        env = gym.make('CartPole-v0')

        scorer = evaluate_on_environment(env)

        cql = CQL()

        mean_episode_return = scorer(cql)


    Args:
        env (gym.Env): gym-styled environment.
        n_trials (int): the number of trials.
        epsilon (float): noise factor for epsilon-greedy policy.
        render (bool): flag to render environment.

    Returns:
        callable: scoerer function.


    """

    def scorer(algo, *args):
        print('Evaluating online for {} episodes.'.format(n_trials))
        episode_rewards = []
        for _ in range(n_trials):
            observation = env.reset()
            episode_reward = 0.0
            while True:
                if np.random.random() < epsilon:
                    action = env.action_space.sample()
                else:
                    action = torch.argmax(algo.predict(torch.tensor(np.array([observation])))).item()
                observation, reward, done, _ = env.step(action)
                episode_reward += reward

                if render:
                    env.render()

                if done:
                    break
            episode_rewards.append(episode_reward)
        return np.mean(episode_rewards), np.std(episode_rewards)

    return scorer

The following is code for training, which was inspired by a mix of the [PyTorch training tutorial](https://pytorch.org/tutorials/beginner/introyt/trainingyt.html) and the [Offline RL tutorial](https://colab.research.google.com/drive/1oJOYlAIOl9d1JjlutPY66KmfPkwPCgEE?usp=sharing).

In [ ]:
def make_target_q_values(model, transition, gamma):
    q_values = model.predict(transition['state']).float()
    q_values_next = model.predict(transition['next state'])
    # to index with action_indices, we need to convert to long data type
    action_indices = transition['action'].long()
    rewards = transition['reward'].float()
    terminals = transition['terminal']
    # detach first and then clone is slightly more efficient than clone first and then detach
    # https://stackoverflow.com/questions/55266154/pytorch-preferred-way-to-copy-a-tensor
    q_values[torch.arange(len(q_values)), action_indices.long()] = rewards.detach().clone()
    max_q_values_next, max_q_values_next_indices = torch.max(q_values_next, dim=1)
    max_q_values_next *= gamma
    q_values[torch.arange(len(q_values)), action_indices] += torch.tensor(
        [0.0 if done else float(max_q_values_next[idx]) for idx, done in enumerate(terminals)])
    return transition['state'], q_values, action_indices


def train_one_epoch(training_loader, model, epoch_index, tb_writer, gamma):
    running_loss = 0.
    last_loss = 0.

    # Here, we use enumerate(training_loader) instead of
    # iter(training_loader) so that we can track the batch
    # index and do some intra-epoch reporting
    for i, transition in enumerate(training_loader):
        # Every transition is {'state', 'action', 'reward', 'next state', 'terminal'}

        # Zero your gradients for every batch!
        model.zero_grad()

        states, target_q_values, action_indices = make_target_q_values(model, transition, gamma)

        loss = model.offline_update(transition['state'], target_q_values, action_indices)

        # Gather data and report
        running_loss += loss
        if i % 10 == 0:
            last_loss = running_loss / 10  # loss per batch
            # print('  batch {} loss: {}'.format(i + 1, last_loss))
            tb_x = epoch_index * len(training_loader) + i + 1
            tb_writer.add_scalar('Loss/train', last_loss, tb_x)
            running_loss = 0.

    return last_loss


def offline_q_learning(model, training_dataset, validation_dataset, max_epochs=100, batch_size=64, gamma=0.9):
    train_loader = DataLoader(training_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
    val_loader = DataLoader(validation_dataset, batch_size=batch_size, shuffle=False, num_workers=2)

    # Initializing in a separate cell so we can easily add more epochs to the same run
    timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
    writer = SummaryWriter('runs/fashion_trainer_{}'.format(timestamp))
    epoch_number = 0

    best_vloss = 1_000_000.

    for epoch in range(max_epochs):
        print('EPOCH {}:'.format(epoch_number + 1))

        # Make sure gradient tracking is on, and do a pass over the data
        model.train(True)
        avg_loss = train_one_epoch(train_loader, model, epoch_number, writer, gamma)

        # We don't need gradients on to do reporting
        model.train(False)

        running_vloss = 0.0
        for i, validation_transition in enumerate(val_loader):
            _, target_q_values, action_indices = make_target_q_values(model, validation_transition, gamma)
            vloss = model.compute_loss(validation_transition['state'], target_q_values, action_indices)
            running_vloss += vloss

        avg_vloss = running_vloss / (i + 1)
        print('LOSS train {} valid {}'.format(avg_loss, avg_vloss))

        # Log the running loss averaged per batch
        # for both training and validation
        writer.add_scalars('Training vs. Validation Loss',
                           {'Training': avg_loss, 'Validation': avg_vloss},
                           epoch_number + 1)
        writer.flush()

        # Track best performance, and save the model's state
        if avg_vloss < best_vloss:
            best_vloss = avg_vloss
            # model_path = 'model_{}_{}'.format(timestamp, epoch_number)
            # torch.save(model.state_dict(), model_path)

        epoch_number += 1
    return model, [avg_loss], [avg_vloss]


Create a class that will assist in quickly loading the data for offline training.

In [ ]:
class CartPoleDataset(Dataset):
    """ 
    Offline reinforcement learning dataset of Cart Pole
    https://pytorch.org/tutorials/beginner/data_loading_tutorial.html#afterword-torchvision
    """

    def __init__(self, data):
        self.dataset = data
        self.transitions, self.unique_states = self.transform_data()

    def __len__(self):
        return len(self.transitions)

    def __getitem__(self, idx):
        return self.transitions[idx]

    def transform_data(self):
        states = []
        transitions = []
        for episode in self.dataset:
            for transition in episode.transitions:
                done = transition.terminal == 1.0
                states.append(list(transition.observation))
                value = {'state': transition.observation, 'action': transition.action, 'reward': transition.reward,
                         'next state': transition.next_observation, 'terminal': done}
                transitions.append(value)
        return transitions, np.array(states)

You can set the seed for reproducibility, define the number of maximum epochs to allow the agent to train, batch size, etc.

The antecedents or the fuzzy logic rules produced by the `unsupervised` function may change due to randomness (there may be more or less, they may behave differently, etc.). 

Note: You may need to do a fresh reinstantiation of the random number generators, environment, and so forth.

In [ ]:
SEED = 39  # used in the 'human-in-the-loop' example
MAX_EPOCHS = 20  # the maximum allowed number of epochs allowed for offline training
BATCH_SIZE = 64  # the number of training observations to sample from the data to perform a single update
CQL_ALPHA = 0.5  # the weight given to the CQL adjustment, a lower value is better when more data is available and vice versa
LEARNING_RATE = 1e-3  # the learning rate used in the paper, later, a better learning rate was found which was 3e-4
NUM_OF_TRAIN_EPISODES = 60  # the amount of training episodes that should be retrieved from CartPoleDataset to do offline training

Add reproducibility by setting the environment seed.

In [ ]:
print('Using seed {}'.format(SEED))
os.environ['PYTHONHASHSEED'] = str(SEED)
torch.manual_seed(SEED)
random.seed(SEED)
np.random.seed(SEED)
env = gym.make('CartPole-v1')
env.seed(SEED)
env.action_space.seed(SEED)

Using seed 39


/usr/local/lib/python3.7/dist-packages/gym/core.py:318: DeprecationWarning: WARN: Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing wrapper in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."
/usr/local/lib/python3.7/dist-packages/gym/wrappers/step_api_compatibility.py:40: DeprecationWarning: WARN: Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future.
  "Initializing environment in old step API which returns one bool instead of two. It is recommended to set `new_step_api=True` to use new step API. This will be the default behaviour in future."
/usr/local/lib/python3.7/dist-packages/gym

[39]

In [ ]:
# Number of states
n_state = env.observation_space.shape[0]
# Number of actions
n_action = env.action_space.n

Apply Fuzzy Conservative Q-Learning algorithm to learn the Q-values.

In [ ]:
dataset, _ = get_cartpole()
dataset = dataset[:1000]
print('num of training episodes available: {}'.format(NUM_OF_TRAIN_EPISODES))
# split train and test episodes
train_episodes, val_episodes = train_test_split(dataset, test_size=0.2)
train_episodes = train_episodes[:NUM_OF_TRAIN_EPISODES]

train_data = CartPoleDataset(train_episodes)
val_data = CartPoleDataset(val_episodes)

# get replay results
rules, _, antecedents = unsupervised(train_data.unique_states, ecm=True, Dthr=4e-1)

print('There are {} rules'.format(len(rules)))

for input_idx, input_variable in enumerate(antecedents):
    print('There are {} antecedent terms for the {}\'th input variable.'.format(len(input_variable), input_idx + 1))

n_outputs = 2
flc = MultiFLC(len(antecedents), n_outputs, antecedents, rules, learning_rate=LEARNING_RATE, cql_alpha=CQL_ALPHA)

flc, train_epoch_losses, val_epoch_losses = offline_q_learning(flc, train_data,
                                                                val_data, MAX_EPOCHS,
                                                                BATCH_SIZE,
                                                                gamma=0.99)

/usr/local/lib/python3.7/dist-packages/h5py/_hl/dataset.py:767: DeprecationWarning: Passing None into shape arguments as an alias for () is deprecated.
  arr = numpy.ndarray(selection.mshape, dtype=new_dtype)
/usr/local/lib/python3.7/dist-packages/gym/envs/registration.py:594: UserWarning: WARN: The environment CartPole-v0 is out of date. You should consider upgrading to version `v1`.
  f"The environment {id} is out of date. You should consider "


num of training episodes available: 60
The shape of the training data is: (3536, 4)

There are 22 rules
There are 4 antecedent terms for the 1'th input variable.
There are 4 antecedent terms for the 2'th input variable.
There are 4 antecedent terms for the 3'th input variable.
There are 5 antecedent terms for the 4'th input variable.
EPOCH 1:
LOSS train 0.8459863810228775 valid 0.8455386044010067
EPOCH 2:
LOSS train 0.8455134077545706 valid 0.8444381541186788
EPOCH 3:
LOSS train 0.8439760767818228 valid 0.8433591693892548
EPOCH 4:
LOSS train 0.8426168971756172 valid 0.8422801936130441
EPOCH 5:
LOSS train 0.8429263288405615 valid 0.8414591213376138
EPOCH 6:
LOSS train 0.8394748415749449 valid 0.8403879644666077
EPOCH 7:
LOSS train 0.8375696922618833 valid 0.8395718607738779
EPOCH 8:
LOSS train 0.8419149581214616 valid 0.83869576582834
EPOCH 9:
LOSS train 0.8393666981754654 valid 0.8378751801329283
EPOCH 10:
LOSS train 0.8371126085198247 valid 0.8371777585846968
EPOCH 11:
LOSS train 0.83

The code to record the performance of the agent is from the [offline RL tutorial](https://colab.research.google.com/github/takuseno/d3rlpy/blob/master/tutorials/cartpole.ipynb#scrollTo=QcVCS0_VaqHr).

In [ ]:
import io
import glob
import base64

from IPython.display import HTML
from gym.wrappers import Monitor
from pyvirtualdisplay import Display
from IPython import display as ipythondisplay

# start virtual display
display = Display(visible=0, size=(1400, 900))
display.start()

# play recorded video
def show_video():
    mp4list = glob.glob('video/*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        ipythondisplay.display(HTML(data='''
            <video alt="test" autoplay loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
            </video>'''.format(encoded.decode('ascii'))))
    else: 
        print("Could not find video")

ImportError: ignored

Test your agent online! (this may take awhile if your agent is performing well)

In [ ]:
# wrap Monitor wrapper
# env = Monitor(env, './video', force=True)

# evaluate
avg_score, std_score = evaluate_on_environment(env)(flc)
print((avg_score, std_score))

Evaluating online for 100 episodes.
(500.0, 0.0)


Watch a video of how it did!

In [ ]:
show_video()